In [1]:
# Chrome의 경우 | 아까 받은 chromedriver의 위치를 지정해준다.
from selenium import webdriver
import requests
from bs4 import BeautifulSoup
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import pandas as pd
from selenium.webdriver.common.action_chains import ActionChains
import numpy as np
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
# expected_conditions는 Selenium 2.26.0 이후 부터 사용 가능합니다.
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
import re 
import os
import pyautogui


In [2]:
#빅카인즈 사이트 접속이 3초 소요된다.
driver = webdriver.Chrome()
driver.implicitly_wait(3)
driver.get('https://www.bigkinds.or.kr/')
#로그인
driver.find_element_by_css_selector('#app-top-navbar > ul > li:nth-child(1) > a').click()
#아이디
driver.find_element_by_css_selector('#login-user-id').send_keys('sda96@naver.com')
#비밀번호
driver.find_element_by_css_selector('#login-user-password').send_keys('5491qwer')
driver.find_element_by_css_selector('#login-btn').click()

In [3]:
def search(word):
    #검색
    #기간 1년으로 설정
    a = WebDriverWait(driver, 30).until(EC.presence_of_element_located((By.CSS_SELECTOR,'#news-search-form > div > div > div > div.main-search-filters.text-left.mt-2')))
    b = a.find_element_by_css_selector('#date-filter-btn')
    time.sleep(1)
    b.click()
    driver.find_element_by_css_selector('#date-filter-div > div > div:nth-child(1) > button.btn.btn-sm.w-100.main-search-filters__dropdown__btn.date-select-btn').click()
    driver.find_element_by_css_selector('#date-confirm-btn').click()
    #검색어 지정
    driver.find_element_by_css_selector('#total-search-key').send_keys(word)
    dd = driver.find_element_by_css_selector('#news-search-form > div > div > div > div.input-group.main-search__form > span > button')
    dd.click()
    time.sleep(4)

In [4]:
#경로를 만들고 만들어진 경로로 저장위치를 변경한다.
def createdir(directory):
    try:
        #경로 생성
        os.makedirs(directory)
        #경로위치 변경
        os.chdir(directory)
    except OSError as e:
        print("Failed to create directory!!!!!")

In [5]:
#기사내용 텍스트파일에 저장
def save_csv(word,code):      
    #해당지점에 기본형이라는 텍스트 파일을 열고 새로운 것을 이어적는다.
    f = open('./기본형_'+word+'.txt','a',encoding='utf-8') 
    f.write(code+"\t")
    f.close()

In [6]:
#기사내용 변형버전 텍스트파일에 저장
def save_csv_tran(word,code):      
    #해당지점에 변형이라는 텍스트 파일을 열고 새로운 것을 이어적는다.
    f = open('./변형_'+word+'.txt','a',encoding='utf-8') 
    f.write(code+"\t")
    f.close()

In [7]:
# 숫자를 입력받아서 한글로 출력하는 함수
# 1~999
def readNumber(n):
    n = int(n)
    units = [''] + list('십백천')
    nums = '일이삼사오육칠팔구'
    result = []
    i = 0
    while n > 0:
        n, r = divmod(n, 10)
        if r > 0:
            result.append(nums[r-1] + units[i])
        i += 1
    return ''.join(result[::-1])

In [8]:
# 1억미만의숫자까지 셀수있게만듬
def readNumber2(n):
    #"""1억미만의 숫자를 읽는 함수"""
    n = int(n)
    a, b = [readNumber(x) for x in divmod(n, 10000)]
    if a:
        return a + "만" +  b
    return b

In [9]:
#숫자를 한글로 변환시켜주는 함수
def digit(a):
    leng = len(re.findall('\d+',a))
    for i in range(0,leng):
        try:
            m = re.search("\d+",a)
            mo = m.group()
            a = re.sub(mo,readNumber2(mo),a)
        #문장중에서 숫자가 포함되지 않는 문장은 a로 그대로 반환시킨다로 오류해결
        except AttributeError:
            return a
    return a

In [10]:
#한글빼고 모두 제거
def cleanText(readData):
    #단어나 숫자가 13자리이상인 것을 공백으로 대체한다
    result = re.sub('[^ ㄱ-ㅣㅣ가-힣|0-9|a-z|A-Z]+','',readData)
    result = re.sub('\w{12,}','',result)
    result = re.sub('div','',result)
    result = re.sub('br','',result)
    result = re.sub('span classquot0','',result)
    result = re.sub('span','',result)
    result = re.sub('classquot','',result)
    result = re.sub('mt2','',result)
    result = re.sub('mb2img','',result)
    result = re.sub('alt','',result)
    result = re.sub('16px','',result)
    result = re.sub('h5','',result)
    return result

In [11]:

#한글빼고 모두 제거 변형버전
def cleanText_tran(readData):
    #단어나 숫자가 13자리이상인 것을 공백으로 대체한다
    result = re.sub('[^ ㄱ-ㅣㅣ가-힣|0-9|a-z|A-Z]+','',readData)
    result = re.sub('\w{12,}','',result)
    result = re.sub('div','',result)
    result = re.sub('br','',result)
    result = re.sub('span classquot0','',result)
    result = re.sub('span','',result)
    result = re.sub('classquot','',result)
    result = re.sub('mt2','',result)
    result = re.sub('mb2img','',result)
    result = re.sub('alt','',result)
    result = re.sub('16px','',result)
    result = re.sub('h5','',result)
    result = re.sub('5G','파이브지',result)
    result = re.sub('5g','파이브지',result)
    result = re.sub('S10','에스텐',result)
    result = re.sub('노트10','노트텐',result)
    result = re.sub('TB','테라바이트',result)
    result = re.sub('GB','기가바이트',result)
    result = re.sub('MB','메가바이트',result)
    result = re.sub("㎡",'제곱미터',result)
    result = re.sub("℃",'도',result)
    result = re.sub("km",'킬로미터',result)
    result = re.sub("mm",'미리미터',result)
    result = re.sub("cm",'센치미터',result)
    result = re.sub('kg','키로그램',result)
    result = re.sub("A",'에이',result)
    result = re.sub("B",'비',result)
    result = re.sub("C",'시',result)
    result = re.sub("D",'디',result)
    result = re.sub("E",'이',result)
    result = re.sub("F",'에프',result)
    result = re.sub("G",'지',result)
    result = re.sub("H",'에이치',result)
    result = re.sub("I",'아이',result)
    result = re.sub("J",'제이',result)
    result = re.sub("K",'케이',result)
    result = re.sub("L",'엘',result)
    result = re.sub("M",'엠',result)
    result = re.sub("N",'엔',result)
    result = re.sub("O",'오',result)
    result = re.sub("P",'피',result)
    result = re.sub("Q",'큐',result)
    result = re.sub("R",'알',result)
    result = re.sub("S",'에스',result)
    result = re.sub("T",'티',result)
    result = re.sub("U",'유',result)
    result = re.sub("V",'브이',result)
    result = re.sub("W",'더블유',result)
    result = re.sub("X",'엑스',result)
    result = re.sub("Y",'와이',result)
    result = re.sub("Z",'제트',result)
    result = re.sub("a",'에이',result)
    result = re.sub("b",'비',result)
    result = re.sub("c",'시',result)
    result = re.sub("d",'디',result)
    result = re.sub("e",'이',result)
    result = re.sub("f",'에프',result)
    result = re.sub("g",'지',result)
    result = re.sub("h",'에이치',result)
    result = re.sub("i",'아이',result)
    result = re.sub("j",'제이',result)
    result = re.sub("k",'케이',result)  
    result = re.sub("l",'엘',result)
    result = re.sub("m",'엠',result)
    result = re.sub("n",'엔',result)
    result = re.sub("o",'오',result)
    result = re.sub("p",'피',result)
    result = re.sub("q",'큐',result)
    result = re.sub("r",'알',result)
    result = re.sub("s",'에스',result)
    result = re.sub("t",'티',result)
    result = re.sub("u",'유',result)
    result = re.sub("v",'브이',result)
    result = re.sub("w",'더블유',result)
    result = re.sub("x",'엑스',result)
    result = re.sub("y",'와이',result)
    result = re.sub("z",'제트',result)
    result = digit(result)
    return result

In [12]:
#해당페이지의 기사 클릭후 문자저장 닫기 
def auto(word):
    for i in range(1,11):
        time.sleep(2)
        try:
            #i번째 기사를 클릭한다 
            csselector = '#news-results > div:nth-child(%s) > div.news-item__body > h4'%i
            WebDriverWait(driver, 30).until(EC.presence_of_element_located((By.CSS_SELECTOR,csselector))).click()
            html = driver.page_source
            soup = BeautifulSoup(html, 'html.parser') 
            ##기사내용 지정후 한글추출이 완료되면 data에 리스트형태로 저장하고 text파일에 추가한다
            te = soup.select('#news-detail-modal > div > div > div.modal-body > div')
            #선택한 부분의 css_selector가 리스트형태이기에 문자형으로 바꾼다.
            ste = str(te)
            ste = cleanText(ste)
            s_ste = str(te)
            s_ste = cleanText_tran(s_ste)
            save_csv(word,ste)
            save_csv_tran(word,s_ste)
            time.sleep(3)
            WebDriverWait(driver, 30).until(EC.presence_of_element_located((By.CSS_SELECTOR,'#news-detail-modal > div > div > div.modal-header > button'))).click()
        #에러가 발생하면 새로고침 후 5초를 기다린다음 다음 기사로 넘어가라
        except Exception as e:
            time.sleep(1)
            pyautogui.click(-100, 200, button='left')
            time.sleep(1)
            pass

In [13]:
#다음페이지를 클릭할때까지 30초 기다리고 클릭이 완료되면 로딩이 길기때문에 20초를 기다린다.
def next_page():
    try:
        WebDriverWait(driver, 30).until(EC.presence_of_element_located((By.CSS_SELECTOR,'#news-results-pagination > ul > li:nth-child(10) > a'))).click()
        time.sleep(20)
    #다음페이지 클릭하였는데 오류발생시 기사에서 나와 다시 다음페이지를 클릭한다.
    except Exception as e:
        pyautogui.click(-100, 200, button='left')
        time.sleep(1)
        WebDriverWait(driver, 30).until(EC.presence_of_element_located((By.CSS_SELECTOR,'#news-results-pagination > ul > li:nth-child(10) > a'))).click()
        time.sleep(20)
        pass

In [14]:
#기사추출 반복문
def bigkinds_crawler(directory,*find):
    createdir(directory)
    # *find에 들어가는 단어들을 차례대로 검색창에 입력한다.
    word_list = [*find]
    word = ""
    for word in word_list:
        search(word)
        time.sleep(5)
        #미구현
        aaa = WebDriverWait(driver, 100).until(EC.presence_of_element_located((By.CSS_SELECTOR,'#news-results-pagination > ul > li:nth-child(11) > a')))
        aaa = re.findall("\d+",aaa.text)
        page_num = int(aaa[0])
        start_code = 0
        while start_code < page_num:
            try:
                start_code = start_code + 1
                code = start_code
                auto(word)
                next_page()
            except:
                time.sleep(30)
                auto(word)
                next_page()
                pass
    #위의 작업이 끝나면 현재 브라우저를 종료한다
    driver.quit()

In [ ]:
bigkinds_crawler("C:/Users/uoykc/Desktop/빅카인즈기사/교육","교육")

In [ ]:
print("*"*50+"연습선"+"*"*50)